# Install and import dependencies

In [ ]:
%pip install tensorflow opencv-python mediapipe scikit-learn matplotlib pandas seaborn

In [1]:
import os
import cv2
import numpy as np
import mediapipe as mp

import pandas as pd
import seaborn as sns
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import load_model

# Face gestures recognition

# Helper functions

In [5]:
# Initializes the holistic model from the MediaPipe library used for detecting and tracking multiple body landmarks, including the face, hands, and pose.
mp_holistic = mp.solutions.holistic 
# Initializes the drawing utilities from the MediaPipe library used to draw the detected landmarks on the image.
mp_drawing = mp.solutions.drawing_utils

def mediapipe_detection(image, model):
    '''
    Processes an image using a MediaPipe model to detect landmarks.
    Args:
        image: The image to process.
        model: The MediaPipe model to use for detection.
    Returns:
        image: The image with the detected landmarks.
        results: The landmarks detected in the image.
    '''
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # COLOR CONVERSION BGR 2 RGB
    image.flags.writeable = False                  # Image is no longer writeable
    results = model.process(image)                 # Make prediction
    image.flags.writeable = True                   # Image is now writeable 
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # COLOR COVERSION RGB 2 BGR
    return image, results

def draw_styled_landmarks(image, results):
    '''
    Draws the detected landmarks on the image.

    Args: 
        image: The image to draw the landmarks on.
        results: The landmarks detected in the image.

    Returns:
        image: The image with the detected landmarks drawn on it.
    '''
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_CONTOURS) # Draw face connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS) # Draw pose connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw right hand connections

def extract_keypoints(results):
    '''
    Extracts the keypoints from the results of the MediaPipe model.

    Args:
        results: The results of the MediaPipe model

    Returns:
        np.array: The keypoints extracted from the results of the MediaPipe model
    '''
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([pose, face, lh, rh])

def prob_viz(res, actions, input_frame, colors):
    '''
    Visualizes the probabilities of the actions detected.
    
    Args:
        res: The probabilities of the actions detected.
        actions: The actions detected.
        input_frame: The frame to draw the visualization on.
        colors: The colors to use for the visualization.
        
    Returns:
        output_frame: The frame with the visualization drawn on it.
    '''
    output_frame = input_frame.copy()
    for num, prob in enumerate(res):
        cv2.rectangle(output_frame, (0,60+num*40), (int(prob*100), 90+num*40), colors[num], -1)
        cv2.putText(output_frame, actions[num], (0, 85+num*40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
        
    return output_frame

# Setup folders

In [6]:
# Path for exported data, numpy arrays
DATA_PATH = os.path.join('Data_2')

# actions that we are detecting
actions = np.array(['hello', 'thanks', 'iloveyou'])

# 30 videos worth of data
no_sequences = 30

# Videos are going to be 30 frames in length
sequence_length = 30


# creating folders
for action in actions:
    for sequence in range(no_sequences):
        try:
            os.makedirs(os.path.join(DATA_PATH, action, str(sequence)))
        except:
            # already exists
            pass

# Collecting Keypoints for training and testing

In [ ]:
cap = cv2.VideoCapture(0) # create class for capturing video from default camera

with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    # Go through each frame of each video of each action
    for action in actions:
        for sequence in range(no_sequences):
            for frame_num in range(sequence_length):
                # Read the video frame
                ret, frame = cap.read()

                # Make detections
                image, results = mediapipe_detection(frame, holistic)

                # Draw landmarks
                draw_styled_landmarks(image, results)

                # display text on the screen
                # start of new video sequence
                if frame_num == 0:
                    cv2.putText(image, 'STARTING COLLECTION', (120, 200), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 4, cv2.LINE_AA)
                    cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15, 12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                    cv2.imshow('OpenCV Feed', image)
                    cv2.waitKey(2000)
                else:
                    cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15, 12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                    cv2.imshow('OpenCV Feed', image)

                # Export keypoints
                keypoints = extract_keypoints(results)
                npy_path = os.path.join(DATA_PATH, action, str(sequence), str(frame_num))
                np.save(npy_path, keypoints)

                if cv2.waitKey(10) & 0xFF == ord('q'):
                    break

    # Release the camera and close the windows
    cap.release()
    cv2.destroyAllWindows()

# Data preprocessing

In [7]:
label_map = {label:num for num, label in enumerate(actions)}

# load and process gesture data from data path
sequences, labels = [], []
for action in actions:
    for sequence in np.array(os.listdir(os.path.join(DATA_PATH, action))).astype(int):
        # loading frames from each sequence
        window = []
        for frame_num in range(sequence_length):
            res = np.load(os.path.join(DATA_PATH, action, str(sequence), f'{frame_num}.npy'))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])

# print(np.array(sequences).shape)
# print(np.array(labels).shape)

X = np.array(sequences)
y = to_categorical(labels).astype(int)

In [18]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=0)

input_shape = (X_train.shape[1], X_train.shape[2])
output_shape = actions.shape[0]

# Train LSTM network

In [ ]:
model = Sequential(
    [
        LSTM(64, return_sequences=True, activation='relu', input_shape=input_shape),
        LSTM(128, return_sequences=True, activation='relu'),
        LSTM(64, return_sequences=False, activation='relu'),
        Dense(64, activation='relu'),
        Dense(32, activation='relu'),
        Dense(output_shape, activation='softmax')
    ]
)

model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])
# model.summary()

In [ ]:
early_stopping = EarlyStopping(monitor='loss', patience=10)
model.fit(X_train, y_train, epochs=1000, callbacks=[early_stopping])

model.save('gesture_model.h5')

# Test the model

In [ ]:
ywhat = model.predict(X_test)

ytrue = np.argmax(y_test, axis=1).tolist()
ywhat = np.argmax(ywhat, axis=1).tolist()

target_names = label_map.keys()

cm = confusion_matrix(ytrue, ywhat)
cm_matrix = pd.DataFrame(data=cm)

sns.heatmap(cm_matrix, annot=True, fmt='d', cmap='viridis')

print(classification_report(ytrue, ywhat, target_names=target_names))
print('Model Accuracy: {}'.format(accuracy_score(ytrue, ywhat)))

# Test in Real Time

In [ ]:
model = load_model('gesture_model.h5')
# used for visualization
colors = [(245,117,16), (117,245,16), (16,117,245)]

# 1. New detection variables
sequence = []
sentence = []
predictions = []
threshold = 0.5

cap = cv2.VideoCapture(0)
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():

        # Read feed
        ret, frame = cap.read()

        # Make detections
        image, results = mediapipe_detection(frame, holistic)
        
        # Draw landmarks
        draw_styled_landmarks(image, results)
        
        # 2. Prediction logic
        keypoints = extract_keypoints(results)
        sequence.append(keypoints)
        sequence = sequence[-sequence_length:]
        
        if len(sequence) == sequence_length:
            res = model.predict(np.expand_dims(sequence, axis=0))[0]
            print(actions[np.argmax(res)])
            predictions.append(np.argmax(res))
            
            
            #3. Viz logic
            # Checks if the most frequent prediction in the last 10 predictions matches the current prediction and it is above the threshold
            if np.unique(predictions[-10:])[0]==np.argmax(res): 
                if res[np.argmax(res)] > threshold: 
                    if len(sentence) > 0: 
                        if actions[np.argmax(res)] != sentence[-1]:
                            sentence.append(actions[np.argmax(res)])
                    else:
                        sentence.append(actions[np.argmax(res)])

            # keeping only last 5 actions in sentence
            if len(sentence) > 5: 
                sentence = sentence[-5:]

            # Viz probabilities
            image = prob_viz(res, actions, image, colors)
            
        cv2.rectangle(image, (0,0), (640, 40), (245, 117, 16), -1)
        cv2.putText(image, ' '.join(sentence), (3,30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        
        # Show to screen
        cv2.imshow('OpenCV Feed', image)

        # Break gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

# Hand gesture recognition

# Helper functions

In [2]:
# Initializes the hands model from the MediaPipe library used for detecting and tracking hand landmarks.
mp_hands = mp.solutions.hands
# Initializes the drawing utilities from the MediaPipe library used to draw the detected landmarks on the image.
mp_drawing = mp.solutions.drawing_utils

def mediapipe_detection(image, model):
    '''
    Processes an image using a MediaPipe model to detect landmarks.
    Args:
        image: The image to process.
        model: The MediaPipe model to use for detection.
    Returns:
        image: The image with the detected landmarks.
        results: The landmarks detected in the image.
    '''
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # COLOR CONVERSION BGR 2 RGB
    image.flags.writeable = False                  # Image is no longer writeable
    results = model.process(image)                 # Make prediction
    image.flags.writeable = True                   # Image is now writeable 
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # COLOR COVERSION RGB 2 BGR
    return image, results

def draw_styled_landmarks(image, results):
    '''
    Draws the detected landmarks on the image.

    Args: 
        image: The image to draw the landmarks on.
        results: The landmarks detected in the image.

    Returns:
        image: The image with the detected landmarks drawn on it.
    '''
    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            mp_drawing.draw_landmarks(image, hand_landmarks, mp_hands.HAND_CONNECTIONS) # Draw hand connections

def extract_keypoints(results):
    '''
    Extracts the keypoints from the results of the MediaPipe model.

    Args:
        results: The results of the MediaPipe model

    Returns:
        np.array: The keypoints extracted from the results of the MediaPipe model
    '''
    if results.multi_hand_landmarks:
        lh = np.array([[res.x, res.y, res.z] for res in results.multi_hand_landmarks[0].landmark]).flatten() if len(results.multi_hand_landmarks) > 0 else np.zeros(21*3)
        rh = np.array([[res.x, res.y, res.z] for res in results.multi_hand_landmarks[1].landmark]).flatten() if len(results.multi_hand_landmarks) > 1 else np.zeros(21*3)
    else:
        lh = np.zeros(21*3)
        rh = np.zeros(21*3)
    return np.concatenate([lh, rh])

def prob_viz(res, actions, input_frame, colors):
    '''
    Visualizes the probabilities of the actions detected.
    
    Args:
        res: The probabilities of the actions detected.
        actions: The actions detected.
        input_frame: The frame to draw the visualization on.
        colors: The colors to use for the visualization.
        
    Returns:
        output_frame: The frame with the visualization drawn on it.
    '''
    output_frame = input_frame.copy()
    for num, prob in enumerate(res):
        cv2.rectangle(output_frame, (0,60+num*40), (int(prob*100), 90+num*40), colors[num], -1)
        cv2.putText(output_frame, actions[num], (0, 85+num*40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
        
    return output_frame

# Setup folders

In [4]:
# Path for exported data, numpy arrays
DATA_PATH = os.path.join('Data_2')

# actions that we are detecting
actions = np.array(['hello', 'thanks', 'iloveyou'])

# 30 videos worth of data
no_sequences = 30

# Videos are going to be 30 frames in length
sequence_length = 30


# creating folders
for action in actions:
    for sequence in range(no_sequences):
        try:
            os.makedirs(os.path.join(DATA_PATH, action, str(sequence)))
        except:
            # already exists
            pass

# Collecting Keypoints for training and testing

In [ ]:
cap = cv2.VideoCapture(0) # create class for capturing video from default camera

with mp_hands.Hands(static_image_mode=False, max_num_hands=2, min_detection_confidence=0.5, min_tracking_confidence=0.5) as hands:
    # Go through each frame of each video of each action
    for action in actions:
        for sequence in range(no_sequences):
            for frame_num in range(sequence_length):
                # Read the video frame
                ret, frame = cap.read()

                # Make detections
                image, results = mediapipe_detection(frame, hands)

                # Draw landmarks
                draw_styled_landmarks(image, results)

                # display text on the screen
                # start of new video sequence
                if frame_num == 0:
                    cv2.putText(image, 'STARTING COLLECTION', (120, 200), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 4, cv2.LINE_AA)
                    cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15, 12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                    cv2.imshow('OpenCV Feed', image)
                    cv2.waitKey(2000)
                else:
                    cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15, 12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                    cv2.imshow('OpenCV Feed', image)

                # Export keypoints
                keypoints = extract_keypoints(results)
                npy_path = os.path.join(DATA_PATH, action, str(sequence), str(frame_num))
                np.save(npy_path, keypoints)

                if cv2.waitKey(10) & 0xFF == ord('q'):
                    break

    # Release the camera and close the windows
    cap.release()
    cv2.destroyAllWindows()

# Data preprocessing

In [ ]:
label_map = {label:num for num, label in enumerate(actions)}

# load and process gesture data from data path
sequences, labels = [], []
for action in actions:
    for sequence in np.array(os.listdir(os.path.join(DATA_PATH, action))).astype(int):
        # loading frames from each sequence
        window = []
        for frame_num in range(sequence_length):
            res = np.load(os.path.join(DATA_PATH, action, str(sequence), f'{frame_num}.npy'))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])

# print(np.array(sequences).shape)
# print(np.array(labels).shape)

X = np.array(sequences)
y = to_categorical(labels).astype(int)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=0)

input_shape = (X_train.shape[1], X_train.shape[2])
output_shape = actions.shape[0]

# Train LSTM network

In [ ]:
model = Sequential(
    [
        LSTM(64, return_sequences=True, activation='relu', input_shape=input_shape),
        LSTM(128, return_sequences=True, activation='relu'),
        LSTM(64, return_sequences=False, activation='relu'),
        Dense(64, activation='relu'),
        Dense(32, activation='relu'),
        Dense(output_shape, activation='softmax')
    ]
)

model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])
# model.summary()

c:\Users\Vahidin\Desktop\annt_workshop_gestures\annt_workshop_gestures_edina\.venv\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [ ]:
early_stopping = EarlyStopping(monitor='loss', patience=10)
model.fit(X_train, y_train, epochs=1000, callbacks=[early_stopping])

model.save('gesture_model.h5')

# Test the model

In [ ]:
ywhat = model.predict(X_test)

ytrue = np.argmax(y_test, axis=1).tolist()
ywhat = np.argmax(ywhat, axis=1).tolist()

target_names = label_map.keys()

cm = confusion_matrix(ytrue, ywhat)
cm_matrix = pd.DataFrame(data=cm)

sns.heatmap(cm_matrix, annot=True, fmt='d', cmap='viridis')

print(classification_report(ytrue, ywhat, target_names=target_names))
print('Model Accuracy: {}'.format(accuracy_score(ytrue, ywhat)))

# Test in Real Time

In [ ]:
model = load_model('gesture_model.h5')
# used for visualization
colors = [(245,117,16), (117,245,16), (16,117,245)]

# 1. New detection variables
sequence = []
sentence = []
predictions = []
threshold = 0.5

cap = cv2.VideoCapture(0)
# Set mediapipe model 
with mp_hands.Hands(static_image_mode=False, max_num_hands=2, min_detection_confidence=0.5, min_tracking_confidence=0.5) as hands:
    while cap.isOpened():

        # Read feed
        ret, frame = cap.read()

        # Make detections
        image, results = mediapipe_detection(frame, hands)
        
        # Draw landmarks
        draw_styled_landmarks(image, results)
        
        # 2. Prediction logic
        keypoints = extract_keypoints(results)
        sequence.append(keypoints)
        sequence = sequence[-sequence_length:]
        
        if len(sequence) == sequence_length:
            res = model.predict(np.expand_dims(sequence, axis=0))[0]
            print(actions[np.argmax(res)])
            predictions.append(np.argmax(res))
            
            # 3. Viz logic
            # Checks if the most frequent prediction in the last 10 predictions matches the current prediction and it is above the threshold
            if np.unique(predictions[-10:])[0] == np.argmax(res): 
                if res[np.argmax(res)] > threshold: 
                    if len(sentence) > 0: 
                        if actions[np.argmax(res)] != sentence[-1]:
                            sentence.append(actions[np.argmax(res)])
                    else:
                        sentence.append(actions[np.argmax(res)])

            # keeping only last 5 actions in sentence
            if len(sentence) > 5: 
                sentence = sentence[-5:]

            # Viz probabilities
            image = prob_viz(res, actions, image, colors)
            
        cv2.rectangle(image, (0,0), (640, 40), (245, 117, 16), -1)
        cv2.putText(image, ' '.join(sentence), (3,30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        
        # Show to screen
        cv2.imshow('OpenCV Feed', image)

        # Break gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()